#Inference file for GreenHawak model

This file contains the workflow for the inerference to reproduce the submission using our state of the aart finetuned YOLO8 model for visual pollution detection or infer on new subsets

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
!pip install ultralytics
!git clone https://github.com/asmayamani/GreenHawk.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.6 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=802fc9d60881e7ab

Cloning into 'GreenHawk'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 19.79 MiB | 7.98 MiB/s, done.


In [10]:
from ultralytics import YOLO
import glob
import pandas as pd
from IPython.display import display, Image
from pathlib import Path    

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)


Mounted at /content/drive


#Make the detection with submitted model

In [6]:
import yaml
config = {
         'train': '/content/drive/MyDrive/Smartathon/TrainF/images/',
         'val': '/content/drive/MyDrive/Smartathon/ValF/images/',
         'nc': 11,
         'names': ['graffiti','faded_signage','potholes','garbage',
                   'construction_road','broken_signage',
                   'bad_streetlight','bad_billboard','sand_on_road','clutter_sidewalk','unkept_facade']}

with open("data.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

In [7]:
#make sure this is the correct path for the text images
test_data = '/content/drive/MyDrive/Smartathon/TestF/images'
weights_path = '/content/GreenHawk/best.pt'


In [ ]:
%cd {HOME}

!yolo task=detect mode=predict source={test_data} model={weights_path} imgsz=640 plots=False  save_txt=True save_conf=True save_crop=False iou=0.5 conf=0.1

/content
Ultralytics YOLOv8.0.11 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Fusing... 
Model summary: 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
image 1/2092 /content/drive/MyDrive/Smartathon/TestF/images/006e04a34f54d116f6aff2bb31cb320c.jpg: 384x640 3 garbages, 403.5ms
image 2/2092 /content/drive/MyDrive/Smartathon/TestF/images/006e70d6a51af3e670bbe712a83aa1c5.jpg: 384x640 2 garbages, 1 unkept_facade, 380.5ms
image 3/2092 /content/drive/MyDrive/Smartathon/TestF/images/0081b25823f23a536c9f1952491db636.jpg: 384x640 1 garbage, 366.1ms
image 4/2092 /content/drive/MyDrive/Smartathon/TestF/images/00b48429c1382cc7d1c72926acedbaa7.jpg: 384x640 3 construction_roads, 1 bad_billboard, 346.6ms
image 5/2092 /content/drive/MyDrive/Smartathon/TestF/images/00b5fc2d3153637e65a35363295e14ba.jpg: 384x640 352.4ms
image 6/2092 /content/drive/MyDrive/Smartathon/TestF/images/00bce25b1d3a3422bb366a12e3e9ac6a.jpg: 384x640 1 graffiti, 3 garbages, 346.8ms
image 7/2092 /content/drive/MyDrive/Smartatho

#Prepare the submission files

In [19]:
names = {'0': 'GRAFFITI', '1': 'FADED_SIGNAGE', '2' : 'POTHOLES', '3':'GARBAGE', '4': 'CONSTRUCTION_ROAD', '5': 'BROKEN_SIGNAGE','6':'' ,'7': 'BAD_BILLBOARD', '8': 'SAND_ON_ROAD', '9' : 'CLUTTER_SIDEWALK' , 
                   '6':'BAD_STREETLIGHT','10':'UNKEPT_FACADE'}

In [20]:
def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    #devide by 2 to match competition expectation
    return [xmax/2,	xmin/2,	ymax/2,	ymin/2]


In [ ]:
#Make sure this is the correct path for the predictions made
inferenceFiles = glob.glob('/content/runs/detect/predict3/labels/*.txt')

In [ ]:
#convert yolo laabels to xml annotation
entries = []
for infPath in inferenceFiles:
  try:
    with open(infPath, 'r') as file2:  # read annotation.txt
        for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
          xml = yolo_to_xml_bbox([float(row[1]),float(row[2]),float(row[3]),float(row[4])],1920,1080)
          entry = [float(row[0]),Path(infPath).name[:-4]+'.jpg',names[row[0]],xml[0],xml[1],xml[2],xml[3]]
          entries.append(entry)

    file2.close()
  except (IOError, OSError) as e:
      print(e)



In [ ]:
#saves the submission
solutionDataFrame = pd.DataFrame(entries,columns= ['class','image_path','name','xmax','xmin','ymax','ymin'])
solutionDataFrame[1:].to_csv('submission2.csv')

In [ ]:
!zip -r /content/predict2.zip /content/runs/detect/predict2
!zip -r /content/predict3.zip /content/runs/detect/predict3


In [ ]:
from PIL import Image, ImageDraw

def draw_image(img, bboxes):
    draw = ImageDraw.Draw(img)
    for bbox in bboxes:
        draw.rectangle(bbox, outline="red", width=2)
    img.save("example.jpg")
    img.show()
